<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_06_Transfer_learning_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [3]:
##Creating Helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-07 17:39:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-07 17:39:49 (78.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
#import series of helper functions for our notebook
from helper_functions import unzip_data,load_and_prep_image, plot_loss_curves, create_tensorboard_callback, compare_historys, walk_through_dir

## 101 Food Classes: working with less data

the data we're downloading is from the original Food101 dataset but with less data samples(10%)

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip 

unzip_data("101_food_classes_10_percent.zip")
train_dir="101_food_classes_10_percent/train"
test_dir="101_food_classes_10_percent/test"


--2023-06-07 17:39:53--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 74.125.137.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   104MB/s    in 21s     

2023-06-07 17:40:15 (73.1 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [6]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/beef_tartare'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/hummus'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/hot_dog'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/edamame'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/dumplings'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/spaghetti_bolognese'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/cheese_plate'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/spaghetti_carbonara'.
There are 0 directories and 250 images in '1

In [7]:
#Setup data inputs
import tensorflow as tf
IMAGE_SIZE=(224,224)
BATCH_SIZE=32

train_data_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                              label_mode="categorical",
                                                                              image_size=IMAGE_SIZE,
                                                                              batch_size=BATCH_SIZE)
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                              image_size=IMAGE_SIZE,
                                                              label_mode="categorical",
                                                              shuffle=False) #Dont shuffle the data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Training the model with transfer learning with 10% of 101 Food Classes

Here are the steps we are going to take
* Setup a model checkpoint callback
* Create a data augmentation layer to build data augmentation right into the model
* Build a headless(no top layers functional efficientNetB0 backboned-model (we'll create our own output layer))
* Compile the model
* Feature extract for 5 full passes(5 epochs on the dataset and validate on 15% of the data of the test data)

In [8]:
#Create checkpoint callback
checkpoint_path="101_classes_10_percent_data_model_checkpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(save_best_only=True,
                                                       filepath=checkpoint_path,
                                                       save_weights_only=True,
                                                       monitor='val_accuracy')

In [9]:
#Create data Augmentation layer to incorporate it right into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation=Sequential([
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    #preprocessing.Rescale(1/255.)# preprocessing built in into EfficientNetB0
],name='data_augmentation')

In [10]:
#Setup the base model and freeze it's layers (feature extraction)
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False

#Setup Model architecture with trainable top layers
inputs=layers.Input(shape=(224,224,3),
                    name="input_layer")
x=data_augmentation(inputs)#augment Images. This only happens during training
x=base_model(x,training=False) #put the base model in inference mode. Weights which needs to stay frozen will stay frozen

x= layers.GlobalAveragePooling2D(name="global_pool_layer")(x)

outputs=layers.Dense(len(train_data_all_10_percent.class_names),
                    activation='softmax',
                    name='output_layer')(x)

model=tf.keras.Model(inputs,outputs)

16705208/16705208 [==============================] - 0s 0us/step


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_pool_layer (GlobalAv  (None, 1280)             0         
 eragePooling2D)                                                 
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainab

In [12]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
history_all_classes_10_percent=model.fit(train_data_all_10_percent,
                                                   steps_per_epoch=len(train_data_all_10_percent),
                                                   epochs=5,
                                                    validation_data=test_data,
                                         validation_steps=int(0.15*len(test_data)),
                                         callbacks=[checkpoint_callback])

Epoch 1/5
237/237 [==============================] - 1121s 5s/step - loss: 3.3699 - accuracy: 0.2747 - val_loss: 2.4734 - val_accuracy: 0.4640


In [ ]:
fine_tuning_results=model.evaluate(test_data)
fine_tuning_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

Looking at the curves, as the 2 curves should be very close to each other, which leads to situation where the model is overfitting

## Fine-tuning

In [ ]:
#Unfreeze all of the layers in the base model 
base_model.trainable = True

#Refreeze the layers except last 5
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#what layers in the model are trainable
for layer in model.layers:
  print(layer.name, layer.trainable)

In [ ]:
for layer in model.layers[2].layers:
  print(layer.name, layer.trainable)

In [ ]:
#Fine Tune for 5 epochs
fine_tune_epochs=10 #model has already done 5 epochs feature extraction
initial_epochs=5

history_all_classes_10_percent_fine_tune=model.fit(train_data_all_10_percent,
                                                   epochs=fine_tune_epochs,
                                                   validation_data=test_data,
                                                   validation_steps=int(0.15*len(test_data)),
                                                   initial_epoch=history_all_classes_10_percent.epoch[-1])

In [ ]:
all_classes_10_percent_fine_tune_results=model.evaluate(test_data)

In [ ]:
plot_loss_curves(history_all_classes_10_percent_fine_tune)

In [ ]:
#compare the histories of feature extraction with fine-tuned model
compare_historys(original_history=history_all_classes_10_percent,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)


## Saving and loading the model

To use the model in an extrnal app, we need to save and export it

In [ ]:
#Save the model
#model.save("101_food_classes_model_fine_tuned_10_percent_data")

In [ ]:
#Load and evaluate the saved model
# loaded_model =tf.keras.models.load_model("101_food_classes_model_fine_tuned_10_percent_data")

In [ ]:
# loaded_model.evaluate(test_data)

Let's make some predictions and vizualize them and see which ones were the most wrong

In [ ]:
import tensorflow as tf

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [ ]:
unzip_data("06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
model=tf.keras.models.load_model("06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
results_downloaded_model=model.evaluate(test_data)

In [ ]:
all_classes_10_percent_fine_tune_results

In [ ]:
results_downloaded_model

## Making predictions with our trained model

In [ ]:
preds_prob=model.predict(test_data, verbose=1) #set verbose to see how long is left

In [ ]:
len(test_data)

In [ ]:
len(preds_prob)

In [ ]:
preds_prob.shape

In [ ]:
#Let's see what the first 10 predictions look like
preds_prob[10]

In [ ]:
preds_prob[0], len(preds_prob[0])

In [ ]:
import numpy as np
np.argmax(preds_prob[0])

In [ ]:
train_data_all_10_percent.class_names[52]

Our model outputs a prediction probabulity array (with N number of variables , where N is the number of classes ) for each sample passed to the prediction method

In [ ]:
print(f"Number of prediction probabilities {len(preds_prob[0])}")
print(f"What prediction probability sample 0 looks like:{preds_prob[0]}" )
print(f"the hishgest value (predicted probab) by the model{preds_prob[0].argmax()}")

In [ ]:
#Get the pred classes of each label
pred_classes=preds_prob.argmax(axis=1)

#How do they look
pred_classes[:10]

In [ ]:
#How many pred_classes do we have
len(pred_classes)

Now we've got a predictions array of all our models predictions, to evaluate them, we need to compare them to the labels

In [ ]:
#To get our test labels, we need to unravel our test_data BatchDataset

y_labels=[]
for images , labels in test_data.unbatch(): #from 790 batches to 25250 images
  y_labels.append(labels.numpy().argmax()) #We want the index value where 1 appears
y_labels[:10]

In [ ]:
#How many y_labels are there
len(y_labels)

## Evaluating our model's predictions

one way to check that our model's prediction array is in the same order as our test labels araay is to find the accuracy score

In [ ]:
results_downloaded_model

In [ ]:
#Let's try sklearn's accuracy score and see what it comes up with

In [ ]:
from sklearn.metrics import accuracy_score
sklearn_accuracy=accuracy_score(y_labels, 
                                y_pred=pred_classes)
sklearn_accuracy

In [ ]:
#Does this metric comes close to our models evaluate results
import numpy as np
np.isclose(results_downloaded_model[1],sklearn_accuracy)

In [ ]:
#Let's create a Confusion Matrix
from helper_functions import make_confustion_matrix

In [ ]:
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=test_data.class_names,
                      figsize=(100,100),
                      test_size=20)

In [ ]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
from sklearn.metrics import confustion_matrix

We need to make some changes to make sure that the titles are printed vetically

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  #Changed( plot-x-labels vertically)
  plt.xticks(rotation=70, font_size=text_size)
  plt.yticks(font_size=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=test_data.class_names,
                      figsize=(100,100),
                      test_size=20)